# Training a Message Passing GNN on a flow past a cylinder

We start by cloning the repo, and installing the dependencies.

In [ ]:
!git clone https://API_KEY@github.com/DonsetPG/graph-physics.git

In [ ]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

In [ ]:
!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

In [ ]:
!pip install loguru==0.7.2
!pip install autoflake==2.3.0
!pip install pytest==8.0.1
!pip install meshio==5.3.5
!pip install tensorflow
!pip install h5py==3.10.0

In [ ]:
!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.4/cu124/repo.html

In [ ]:
!pip install pyvista lightning==2.5.0 wandb "wandb[media]"

In [ ]:
!pip install pytorch-lightning==2.5.0 torchmetrics==1.6.3

## Weights and Biases

Make sure you are connected. If you don't have the webapp open, input your api key and press enter at the next cell.

In [ ]:
import wandb
wandb.login()

# Dataset

We now download the cylinder flow dataset, as `.h5` matrices.

In [ ]:
sh = """
BASE_URL="https://storage.googleapis.com/large-physics-model/datasets/cylinder/"
OUTPUT_DIR="h5_dataset/cylinder_flow"
mkdir -p ${OUTPUT_DIR}

for file in train.h5 test.h5
do
wget -O "${OUTPUT_DIR}/${file}" "${BASE_URL}${file}"
done
"""
with open('script.sh', 'w') as file:
  file.write(sh)

!bash script.sh

Finally, we place the dataset inside of our repo.

In [ ]:
!mkdir graph-physics/dataset/

In [ ]:
!mv h5_dataset/ graph-physics/dataset/

# Pyvista

When running Pyvista vizualisation in Google Colab, we need to add some packages

In [ ]:
!apt-get install -qq xvfb
!pip install pyvista panel -q

To make sure they work, we need to run the following code:

```python
import os
os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

import panel as pn
pn.extension('vtk')
```

on top of our training script.

# Training Script

By default, we use the following parameters:

```
python -m graphphysics.train \
            --training_parameters_path=training.json \
            --num_epochs=5 \
            --init_lr=0.001 \
            --batch_size=2 \
            --warmup=500 \
            --num_workers=0 \
            --prefetch_factor=0 \
            --model_save_path=model.ckpt \
            --no_edge_feature
```

Here, since we train a message passing model on an `.h5` dataset, we need to make sure that:
- `--num_workers` is set to 0
- `--prefetch_factor=0` is set to 0
- we remove the `--no_edge_feature` flag

We can udpate the `train.sh` file to use the right training file by moving the file `cylinder.json` from `training_config/` to our main folder and use:

```
python -m graphphysics.train \
            --training_parameters_path=cylinder.json \
            --num_epochs=5 \
            --init_lr=0.001 \
            --batch_size=2 \
            --warmup=500 \
            --num_workers=0 \
            --prefetch_factor=0 \
            --model_save_path=model.ckpt \
            --no_edge_feature
```

In [ ]:
import os
os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

import panel as pn
pn.extension('vtk')
!cd graph-physics && sh train.sh

<ipython-input-8-ba1dc3213b56>:6: UserWarning: Using Panel interactively in Colab notebooks requires the jupyter_bokeh package to be installed. Install it with:

    !pip install jupyter_bokeh

and try again.
  pn.extension('vtk')


DGL backend not selected or invalid.  Assuming PyTorch for now.
Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
2024-10-13 20:26:06.131 | INFO     | __main__:main:54 - Opening training parameters from cylinder.json
2024-10-13 20:26:06.135 | INFO     | __main__:main:146 - Initializing new model
EncodeProcessDecode(
  (nodes_encoder): Sequential(
    (0): Linear(in_features=11, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=128, bias=True)
    (7): RMSNorm()
  )
  (edges_encoder): Sequential(
    (0): Linear(in_features=3, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU

# Final Vizualisation

During training, you can also check some meshes inside of `graph-physics/meshes/epoch_{n}`

At the end of the training, we can zip it to make it easier to download it.

In [ ]:
!zip -r cylinder.zip graph-physics/meshes/epoch_10/